<a href="https://colab.research.google.com/github/gbessardon/Land_Cover_comparison/blob/main/ESRI2020.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Running environment options

In [16]:
colab=1 # 1 running on google collab
drive=1 # 1 need to mount google drive

## ESRI 2020 folder location

In [17]:
ESRIpath='/content/drive/MyDrive/ESRI2020'

## LUCAS file location

In [18]:
fnshp='/content/drive/MyDrive/LUCAS2018/LUCAS_2018_Copernicus_polygons.shp'
fncsv='/content/drive/MyDrive/LUCAS2018/LUCAS_2018_Copernicus_attributes.csv'

# Run on collab and drive 
set colab=1 and drive=1  in running enviroment options

## MOUNT google drive

In [19]:
if drive==1:
  from google.colab import drive
  drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## install geopandas and pyrpoj for collab

In [20]:
if colab==1:
  !pip install geopandas
  !pip install pyproj

# import libraires

In [21]:
import os
from osgeo import gdal
from shapely import geometry
import geopandas as gpd
import numpy as np
import shutil

# Create functions

## get a wgs84 polygon at the extent of the map


In [22]:
def extract_polygon(fn):
  opt=gdal.InfoOptions(format='json')
  info=gdal.Info(fn,options=opt)
  bounding_points=info['wgs84Extent']['coordinates'][0]
  return geometry.Polygon(bounding_points)

## create directory to store the segment before zipping

In [23]:
def create_sgement_dir(path=os.getcwd()):
    outputdir=os.path.join(path,'LUCAS_Segment')
    if not os.path.isdir(outputdir):
      os.mkdir(outputdir)
    return outputdir

## Search for LUCAS POINT ID inside each Each ESRI tile and store it inside the ESRI  GeoDataFrame  


In [24]:
def point_id_cores(ESRIgdf,LUCASgdf):
    pointidvalid=[]
    for eg in ESRIgdf.geometry:
      oklist=[eg.covers(lg) for lg in LUCASgdf.geometry]
      pointidvalid.append(LUCASgdf['POINT_ID'].loc[np.array(oklist)==True].values)
    pointidvalid
    return(pointidvalid)

# Main

## Create a GeoDataFrame with a bounding list of each ESRI tile

In [25]:

tiflist=[os.path.join(ESRIpath,f) for f in os.listdir(ESRIpath) if f.endswith('.tif')]
boundinglist=[extract_polygon(t) for t in tiflist]
d={'filemame': tiflist, 'geometry' : boundinglist}
ESRIgdf=gpd.GeoDataFrame(d,crs="EPSG:4326")

## Open LUCAS shapefile in a geodatabase

In [26]:
LUCASgdf = gpd.read_file(fnshp)

## Search for LUCAS POINT ID inside each Each ESRI tile and store it inside the ESRI  GeoDataFrame  




In [27]:
ESRIgdf['validlucas']=point_id_cores(ESRIgdf,LUCASgdf)

## Create directory to store the segment before zipping

In [28]:
outputdir=create_sgement_dir(path=os.getcwd())

## Create ESRI segments corresponding to the LUCAS segments

In [30]:
for i,d in ESRIgdf.iterrows():
  if len(ESRIgdf['validlucas'][i])>0:
    for j,_ in enumerate(ESRIgdf['validlucas'][i]):
      lucasid=ESRIgdf['validlucas'][i][j]
      outfile=os.path.join(outputdir,str(lucasid)+'.tif')
      lucasp=LUCASgdf['geometry'].loc[LUCASgdf['POINT_ID']==ESRIgdf['validlucas'][i][j]].values[0]
      bounds=lucasp.bounds
      opt=gdal.WarpOptions(dstSRS="EPSG:4326",outputBounds=bounds,
                           targetAlignedPixels=True,xRes=0.000089,yRes=0.000089)
      gdal.Warp(outfile,ESRIgdf.filemame[0],options=opt)

## Save in a zip archive in google drive (prevents I/O errors)

In [31]:
shutil.make_archive(os.path.join(ESRIpath,'LUCAS_Segment'), 'zip', outputdir)

'/content/drive/MyDrive/ESRI2020/LUCAS_Segment.zip'